In [1]:
import os
import sys
import sqlite3
import pandas as pd
import numpy as np
import re

In [2]:
# 제무제표 항목명과 DB 필드명 매핑
dic_reports = {
    '코드': 'code',
    '종목코드': 'code',
    '자산총계': 'total_assets',
    '유동자산': 'current_assets',
    '부채총계': 'total_liabilities',
    '자본총계': 'total_equity',
    '매출액': 'net_sales',
    '매출원가': 'cost_of_sales',
    '매출총이익': 'gross_profit',
    '발표영업이익': 'operating_income',
    '영업이익': 'operating_income',
    '순이익': 'ongoing_operating_income',
    '당기순이익': 'ongoing_operating_income',
    '반기순이익': 'ongoing_operating_income',
    '분기순이익': 'ongoing_operating_income',
    '영업현금흐름': 'cash_flows_from_operatings',
    '유동비율': 'current_ratio',
    '부채비율': 'debt_to_equity_ratio'
}

dic_companies = {
    '코드': 'code',
    '종목코드': 'code',
    '시장': 'market',
    '회사명': 'company',
    '종목명': 'company',
    '산업코드': 'ind_code',
    '산업명': 'industry'
}
# tuple(dic_reports.keys()) + tuple(dic_companies.keys())
# print(type(dic_reports.keys()))

In [3]:
## 201109~201809 까지 유효
yyyy = '2017'
mm = '12'

# 해당 분기 폴더에 대상 파일 리스트 가져옴 (11년 실적월 별 파일 여러개임)
f_path = '../data/financial_reports/%s년%s월확정실적'%(yyyy, mm)
files = [f for f in os.listdir(f_path) if os.path.isfile(os.path.join(f_path, f))]
files = [f for f in files if 'Eng' not in f and 'Rank' not in f and 'ENG' not in f and '영문' not in f and 'hwp' not in f and 'txt' not in f]
files

# 일단 첫번째 파일만 처리 (여러파일 처리는 따로..)
report_list = pd.read_excel('%s/%s'%(f_path, files[0]), sheet_name=None)
print(report_list.keys())

# 여러개 탭 처리하게 수정 필요 (일단 제조-연결만 처리)
reports = report_list['제조(연결)']
# reports = report_list['제조(별도-개별)']
reports.columns = [str.strip(re.sub('\n', '', t)) for t in reports.keys()]
print(reports.keys())

# 필요한 필드만 추출
# cols = ('시장', '코드', '회사명', '산업명', '자산총계', '유동자산', '자본총계', '부채총계', '매출액', '매출원가', '매출총이익',
#         '발표영업이익', '순이익', '분기순이익', '당기순이익', '반기순이익', '영업활동으로인한현금흐름', '부채비율', '유동비율', '자기자본비율')
cols = tuple(dic_reports.keys()) + tuple(dic_companies.keys())
reports = reports[[k for k in reports.keys() if k.startswith(cols) and '3개월' not in k]]
cols = [k for k in reports.keys() if '(' not in k]
cols = cols + [k for k in reports.keys() if '%s%s'%(yyyy, mm) in k]
reports = reports[cols]
print(cols)
reports['코드'] = reports['코드'].str[1:]
reports = reports[~reports['회사명'].str.contains('스팩')]
reports = reports[~reports['회사명'].str.endswith('우')]
reports = reports[~reports['회사명'].str.endswith('우B')]

# 필드명 정제
cols = reports.keys()
cols = [re.sub('누적', '', k) for k in cols] # (보고서기재) 제거
cols = [re.sub('\(%s%s\)'%(yyyy, mm), '', k) for k in cols]
# cols = [re.sub('^.*순이익', '순이익', k) for k in cols]
reports.columns = cols
reports_length = len(reports)
print(reports_length)

dict_keys(['제조(별도-개별)', '제조(연결)', '금융(별도-개별)', '금융(연결)'])
Index(['코드', '회사명', '시장', '업종', '산업명', '매출액(201712)3개월', '매출액(201712)누적',
       '매출액(201709)3개월', '매출액(201709)', '매출액(201706)',
       ...
       '자본총계(201712)누적', '자본총계(201709)', '자본총계(201706)', '자본총계(201703)',
       '자본총계(201612)', '지배주주지분(201712)누적', '지배주주지분(201709)', '지배주주지분(201706)',
       '지배주주지분(201703)', '지배주주지분(201612)'],
      dtype='object', length=182)
['코드', '회사명', '시장', '산업명', '발표영업이익률', '순이익률', '매출액(201712)누적', '매출원가(201712)누적', '매출총이익(201712)누적', '발표영업이익(201712)누적', '순이익(201712)누적', '유동비율(201712)', '부채비율(201712)', '자산총계(201712)누적', '유동자산(201712)누적', '부채총계(201712)누적', '자본총계(201712)누적']
1496


In [4]:
reports.head(1)

,코드,회사명,시장,산업명,발표영업이익률,순이익률,매출액,매출원가,매출총이익,발표영업이익,순이익,유동비율,부채비율,자산총계,유동자산,부채총계,자본총계
0,000040,케이알모터스(주),거래소,기타 운송장비 제조업,-62.483075,-75.611894,41661.895,50298.745,-8636.85,-26031.633,-31501.348,96.713002,139.804254,108443.184,43267.628,63221.641,45221.543


In [5]:
reports.keys()

Index(['코드', '회사명', '시장', '산업명', '발표영업이익률', '순이익률', '매출액', '매출원가', '매출총이익',
       '발표영업이익', '순이익', '유동비율', '부채비율', '자산총계', '유동자산', '부채총계', '자본총계'],
      dtype='object')

In [6]:
# company 기본정보 추출
companies = reports[['코드', '시장', '회사명', '산업명']]
companies.columns = ['code', 'market', 'company', 'industry']
companies.head()
companies_length = len(companies)

In [7]:
# 인서트 전 row 수
conn = sqlite3.connect("../database/quantative_investing.db")
cur = conn.cursor()
cur.execute("select count(*) from companies")
rows = cur.fetchall()
cur.close()
conn.close()
companies_db_before = rows[0][0]
print(rows)

[(0,)]


In [8]:
# company 정보 DB Insert
conn = sqlite3.connect("../database/quantative_investing.db")
for idx, company in companies.iterrows():
    columns = ', '.join(company.keys())
    placeholders = ', '.join('?' * len(company.values))
    sql = 'INSERT INTO companies ({}) VALUES ({})'.format(columns, placeholders)
    
    try:
        conn.execute(sql, company.values)
    except sqlite3.IntegrityError:
        pass # 이미 인서트되어 발생하는 오류 무시
    except:
        print(sys.exc_info())
        pass
conn.commit()
conn.close()

In [9]:
conn = sqlite3.connect("../database/quantative_investing.db")
cur = conn.cursor()
cur.execute("select count(*) from companies")
rows = cur.fetchall()
cur.close()
conn.close()
companies_db_after = rows[0][0]
print(rows)

[(0,)]


In [10]:
reports.keys()

Index(['코드', '회사명', '시장', '산업명', '발표영업이익률', '순이익률', '매출액', '매출원가', '매출총이익',
       '발표영업이익', '순이익', '유동비율', '부채비율', '자산총계', '유동자산', '부채총계', '자본총계'],
      dtype='object')

In [11]:
# DB 필드명과 동일하게 DF 필드명 변경
keys = reports.keys()
print(list(keys))
keys2 = [dic_reports[k] if k in dic_reports else k for k in keys ]
print(keys2)
reports.columns = keys2
reports = reports[[k for k in reports.keys() if k in dic_reports.values()]]
reports.head()

['코드', '회사명', '시장', '산업명', '발표영업이익률', '순이익률', '매출액', '매출원가', '매출총이익', '발표영업이익', '순이익', '유동비율', '부채비율', '자산총계', '유동자산', '부채총계', '자본총계']
['code', '회사명', '시장', '산업명', '발표영업이익률', '순이익률', 'net_sales', 'cost_of_sales', 'gross_profit', 'operating_income', 'ongoing_operating_income', 'current_ratio', 'debt_to_equity_ratio', 'total_assets', 'current_assets', 'total_liabilities', 'total_equity']


,code,net_sales,cost_of_sales,gross_profit,operating_income,ongoing_operating_income,current_ratio,debt_to_equity_ratio,total_assets,current_assets,total_liabilities,total_equity
0,000040,41661.895,50298.745,-8636.850,-26031.633,-31501.348,96.713002,139.804254,108443.184,43267.628,63221.641,45221.543
1,000050,360788.750,253442.168,107346.582,42309.232,25315.361,28.250804,82.231461,1274042.237,90254.697,574908.164,699134.073
2,000070,2396290.235,1974916.010,421374.225,98828.147,67186.351,208.171413,68.965299,3261056.035,1132120.433,1331040.788,1930015.247
3,000080,1889910.216,1086067.409,803842.808,87249.749,12735.966,64.720007,188.096027,3507595.838,1110999.896,2290086.565,1217509.273
4,000100,1462248.030,1037623.911,424624.119,88709.620,109636.659,387.064773,30.401596,2094673.424,1069512.906,488348.428,1606324.995


In [12]:
reports.insert(1, 'rdate', '%s%s31'%(yyyy, mm))
reports.head()

,code,rdate,net_sales,cost_of_sales,gross_profit,operating_income,ongoing_operating_income,current_ratio,debt_to_equity_ratio,total_assets,current_assets,total_liabilities,total_equity
0,000040,20171231,41661.895,50298.745,-8636.850,-26031.633,-31501.348,96.713002,139.804254,108443.184,43267.628,63221.641,45221.543
1,000050,20171231,360788.750,253442.168,107346.582,42309.232,25315.361,28.250804,82.231461,1274042.237,90254.697,574908.164,699134.073
2,000070,20171231,2396290.235,1974916.010,421374.225,98828.147,67186.351,208.171413,68.965299,3261056.035,1132120.433,1331040.788,1930015.247
3,000080,20171231,1889910.216,1086067.409,803842.808,87249.749,12735.966,64.720007,188.096027,3507595.838,1110999.896,2290086.565,1217509.273
4,000100,20171231,1462248.030,1037623.911,424624.119,88709.620,109636.659,387.064773,30.401596,2094673.424,1069512.906,488348.428,1606324.995


In [13]:
len(reports)

1496

In [14]:
conn = sqlite3.connect("../database/quantative_investing.db")
cur = conn.cursor()
cur.execute("select count(*) from reports")
rows = cur.fetchall()
cur.close()
conn.close()
reports_db_before = rows[0][0]
print(reports_db_before)

0


In [15]:
# company 정보 DB Insert
conn = sqlite3.connect("../database/quantative_investing.db")
for idx, report in reports.iterrows():
    columns = ', '.join(report.keys())
    placeholders = ', '.join('?' * len(report.values))
    sql = 'INSERT INTO reports ({}) VALUES ({})'.format(columns, placeholders)
    
    try:
        conn.execute(sql, report.values)
    except sqlite3.IntegrityError:
        pass # 이미 인서트되어 발생하는 오류 무시
    except:
        print(sys.exc_info())
        pass
conn.commit()
conn.close()

In [16]:
conn = sqlite3.connect("../database/quantative_investing.db")
cur = conn.cursor()
cur.execute("select count(*) from reports")
rows = cur.fetchall()
cur.close()
conn.close()
reports_db_after = rows[0][0]
reports_db_after

1496

In [17]:
# summary

print('companies_length : %s'%companies_length)
print('companies_db_before : %s'%companies_db_before)
print('companies_db_after : %s (%s added)'%(companies_db_after, companies_db_after - companies_db_before))
print('reports_length : %s'%reports_length)
print('reports_db_before : %s'%reports_db_before)
print('reports_db_after : %s (%s added)'%(reports_db_after, reports_db_after - reports_db_before))


companies_length : 1496
companies_db_before : 0
companies_db_after : 0 (0 added)
reports_length : 1496
reports_db_before : 0
reports_db_after : 1496 (1496 added)
